**For a detailed explanation of the solution, please see the README.md file**

We start by installing the required libraries

In [ ]:
pip install boto3

We will first create a boto3 bedrock client. We can use this client to issue API calls to Generative AI models available in Bedrock.

Note: You can replace the profile_name with the profile name that is configured on your developer environment that has access to Bedrock.

In [ ]:
from botocore.config import Config
import json
import boto3
REGION_NAME = 'us-east-1'
retry_config = Config(
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
session = boto3.Session(region_name=REGION_NAME, profile_name='default')
bedrock_client = session.client(
        service_name='bedrock-runtime',
        config=retry_config
    )

print("boto3 Bedrock client successfully created!")

Next, we create a resuable function that uses the client we created above to call Claude 3 Sonnet model on Bedrock. We can pass prompt and temperature to this function.

In [ ]:
def call_bedrock_claude_3(prompt_text, temperature):
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [
                        {
                            "type": "text",
                            "text": prompt_text
                        }
                ]
            }
        ]
    }
    body = json.dumps(body)
    response = bedrock_client.invoke_model(
            body=body, modelId=model_id
        )
    # Parse the response
    response_lines = response['body'].readlines()
    json_str = response_lines[0].decode('utf-8')
    json_obj = json.loads(json_str)
    result_text = json_obj['content'][0]['text']
    
    return {'role': 'assistant', 'content': result_text}

Let's test the access to Bedrock with a  sample api call

In [ ]:
# Using Claude 3 Sonnet model
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
prompt = "Hello..."

# Call Bedrock and get the response
response = call_bedrock_claude_3(prompt, 0.7)
print(response)

#### **DynamoDB and Redis**


In this architecture, we use an Amazon DynamoDB table to persistently store all the chat messages. We create one table called "ChatHistory" with "UserId" as the partition key and "Timestamp" as the sort key.

**Creating DynamoDB Table**

In [ ]:
import boto3


TABLE_NAME = 'ChatHistory'

# Initialize the DynamoDB client
dynamodb = boto3.client('dynamodb', region_name=REGION_NAME)

# Create a DynamoDB table
def create_dynamodb_table():
    table_name = TABLE_NAME
    try:
        response = dynamodb.create_table(
            TableName=table_name,
            KeySchema=[
                {"AttributeName": "UserId", "KeyType": "HASH"},  # Partition key
                {"AttributeName": "Timestamp", "KeyType": "RANGE"}  # Sort key
            ],
            AttributeDefinitions=[
                {"AttributeName": "UserId", "AttributeType": "S"},  
                {"AttributeName": "Timestamp", "AttributeType": "S"}, 
            ],
            ProvisionedThroughput={
                "ReadCapacityUnits": 5,
                "WriteCapacityUnits": 5
            }
        )
        print(f"Creating table {table_name}...")
        dynamodb.get_waiter('table_exists').wait(TableName=table_name)
        print(f"Table {table_name} has been created.")
    except Exception as e:
        print(f"Error creating table: {e}")

# Call the function to create the table 
create_dynamodb_table()

Next, we install redis python package that will allow us to read or write from the cache. 

Note: This sample assumes there's a Redis cache already created and you have access to the endpoint. You can use AWS Elasticache to easily create serverless Redis Caches. Please visit this url to learn more: https://aws.amazon.com/blogs/aws/amazon-elasticache-serverless-for-redis-and-memcached-now-generally-available/

Note: If you are using Elasticache Redis Serverless, you have to run this sample from the same VPC as the cache is in.

In [ ]:
pip install redis

In the next block we create two handy functions to test the connection to Redis and DynamoDB. We will add some test data and try retrieving.

In [ ]:
import redis
import boto3
import json
from datetime import datetime

# Configuration for Redis and DynamoDB

redis_host = "<REPLACE_WITH_REDIS_ENDPOINT>"
redis_port = "<REPLACE_WITH_REDIS_PORT>"
dynamodb_table = TABLE_NAME
aws_region = REGION_NAME

# Initialize Redis client
redis_client = redis.Redis(host=redis_host, port=redis_port, decode_responses=True, ssl=True)

# Initialize DynamoDB client
dynamodb_client = boto3.client('dynamodb', region_name=REGION_NAME)


# Test function for Redis connection
def test_redis_connection():
    try:
        # Add a test message to Redis
        user_id = "test_user_redis"
        message_text = "Hello from Redis!"
        timestamp = datetime.utcnow().isoformat()
        message = {"content": message_text, "timestamp": timestamp}
        stack_key = f"{user_id}:stack"
        redis_client.rpush(stack_key, json.dumps(message))

        # Retrieve and print the messages
        retrieved_messages = redis_client.lrange(stack_key, 0, -1)
        print(f"Retrieved messages from Redis for {user_id}:")
        for msg in retrieved_messages:
            print(json.loads(msg))
    except Exception as e:
        print(f"Error in Redis connection test: {e}")


# Test function for DynamoDB connection
def test_dynamodb_connection():
    try:
        # Add a test message to DynamoDB
        user_id = "test_user_dynamodb"
        message_text = "Hello from DynamoDB!"
        timestamp = datetime.utcnow().isoformat()
        dynamodb_client.put_item(
            TableName=TABLE_NAME,
            Item={
                "UserId": {"S": user_id},
                "Timestamp": {"S": timestamp},
                "Content": {"S": message_text},
                "BatchId": {"N": "1"}
            }
        )

        # Query and print the messages
        response = dynamodb_client.query(
            TableName=dynamodb_table,
            KeyConditionExpression="UserId = :user",
            ExpressionAttributeValues={":user": {"S": user_id}}
        )
        print(f"Retrieved messages from DynamoDB for {user_id}:")
        for item in response['Items']:
            print({
                "UserId": item["UserId"]["S"],
                "Timestamp": item["Timestamp"]["S"],
                "Content": item["Content"]["S"],
                "BatchId": item["BatchId"]["N"]
            })
    except Exception as e:
        print(f"Error in DynamoDB connection test: {e}")


# Run the test functions
test_redis_connection()
test_dynamodb_connection()


For testing purposes, we can also create two utility functions to completely cleanup data from Redis and DynamoDB. 

**Please note:** Use these functions with caution, only while testing. These can delete important data in production environments.

In [ ]:
import redis

def clear_all_redis_data(redis_host, redis_port, redis_password=None):
    try:
        # Connect to Redis
        redis_client = redis.Redis(host=redis_host, port=redis_port, decode_responses=True, ssl=True)
        
        # Flush all keys in the current database
        redis_client.flushdb()
        print("Successfully cleared all data in the current Redis database.")
    except Exception as e:
        print(f"Error clearing Redis data: {e}")

# Call the function to clear all data in Redis
clear_all_redis_data(redis_host, 6379, "your-redis-password")  # Remove password if not used

import boto3

def clear_all_dynamodb_data(table_name, aws_region):
    dynamodb_client = boto3.client('dynamodb', region_name=aws_region)
    dynamodb_resource = boto3.resource('dynamodb', region_name=aws_region)
    table = dynamodb_resource.Table(table_name)
    
    try:
        # Scan to get all items
        response = table.scan()
        items = response['Items']

        # Use batch writer to delete items in batches
        with table.batch_writer() as batch:
            for item in items:
                batch.delete_item(Key={"UserId": item['UserId'], "Timestamp": item['Timestamp']})
        
        print(f"Successfully cleared all data from DynamoDB table: {table_name}")
    except Exception as e:
        print(f"Error clearing DynamoDB data: {e}")

# Call the function to clear all data in DynamoDB
clear_all_dynamodb_data(TABLE_NAME, REGION_NAME)





#### **Chat History Summarization**

For every 20 messages in a user's chat, we summarize them using an LLM (Claude 3). The following code creates a simple summarization prompt and a function to call the LLM. You can modify the prompt for your particular use case.

In [ ]:
summarize_prompt = """
Given a history of chat messages, summarize the conversation between user and AI in to one paragraph of not more than 250 words.
Summarize all user messages in to one paragraph and all AI messages in to another paragraph.
User: 
AI: 

Message history:
{conversation}

Instructions:


Summary:
"""

def summarize_chat_history(messages):
    # Combine all user messages (20 messages are passed to this function at a time)
    summary = " ".join([str(msg) for msg in messages])
    # Generate the prompt for summarization
    prompt = summarize_prompt.format(conversation=summary)
    # Call the Bedrock model to generate the summary
    response = call_bedrock_claude_3(prompt, 0.7)
    return response

We now create the ChatManager class which handles the new message workflow, the summarization and chat history retrieval workflows.

Please read inline comments for explanation of each function.

In [ ]:
import redis
import boto3
import json
import threading
from datetime import datetime
import boto3
from boto3.dynamodb.conditions import Key


class ChatManager:
    def __init__(self, redis_host, redis_port, dynamodb_table, region):
        # Connect to Redis
        self.redis_client = redis.Redis(host=redis_host, port=redis_port, decode_responses=True, ssl=True)
        self.lock = threading.Lock()
        
        # Set up DynamoDB client and table
        self.dynamodb_client = boto3.client('dynamodb', region_name=region)
        self.dynamodb_resource = boto3.resource('dynamodb', region_name=region)
        self.dynamodb_table = dynamodb_table
        self.region = region

    def count_messages_with_batch(self, stack_key, batch_id):
        """
        Count the number of messages in the specified stack that have the given batch_id.
        """
        count = 0
        # Retrieve all messages from the Redis stack
        messages = self.redis_client.lrange(stack_key, 0, -1)

        # Count messages with the specified batch_id
        for msg in messages:
            message = json.loads(msg)
            if message["batch_id"] == batch_id:
                count += 1
        return count

    def handle_new_message(self, user_id, message_text):
        with self.lock:
            # Define keys for Redis
            stack_key = f"{user_id}:stack" # Stores individual messages for a particular user
            batch_id_key = f"{user_id}:batch_id" # Stores the current batch ID for a user
            
            # Retrieve the current batch ID from Redis, or initialize it
            batch_id = self.redis_client.get(batch_id_key)
            if batch_id is None:
                batch_id = 1
            else:
                batch_id = int(batch_id)

            # Create a new message dictionary with a timestamp
            timestamp = datetime.utcnow().isoformat()
            message = {"batch_id": batch_id, "content": message_text, "timestamp": timestamp}
            
            # Add the new message to the cache stack for the user. When the stack reaches 20 messages, a summary is created.
            self.redis_client.rpush(stack_key, json.dumps(message))

            # Persist the message to DynamoDB
            self.dynamodb_client.put_item(
                TableName=self.dynamodb_table,
                Item={
                    "UserId": {"S": user_id},
                    "Timestamp": {"S": timestamp},
                    "Content": {"S": message_text},
                    "BatchId": {"N": str(batch_id)}
                }
            )

            # Check if the stack has 20 messages for the current batch ID
            count = self.count_messages_with_batch(stack_key, batch_id)
            if count == 4:
                threading.Thread(target=self._create_summary, args=(user_id, batch_id)).start()
                self.redis_client.set(batch_id_key, batch_id + 1)

    def _create_summary(self, user_id, batch_id):
        with self.lock:
            # Retrieve all messages for the user with the specified batch ID
            stack_key = f"{user_id}:stack"
            all_messages = [json.loads(msg) for msg in self.redis_client.lrange(stack_key, 0, -1)]
            messages = [msg for msg in all_messages if msg["batch_id"] == batch_id]

            # Create a summary of those messages
            summary_content = summarize_chat_history(messages)
            summary = {"batch_id": batch_id, "content": summary_content, "count": len(messages)}

            # Store the summary in Redis
            summary_key = f"{user_id}:summary"
            self.redis_client.set(summary_key, json.dumps(summary))

            # Gather all messages from the cache which have not been summarized yet
            remaining_messages = [msg for msg in all_messages if msg["batch_id"] != batch_id]

            # Clear the stack and repopulate with the remaining messages
            self.redis_client.delete(stack_key)
            for msg in remaining_messages:
                self.redis_client.rpush(stack_key, json.dumps(msg))

    # Get the chat history for a user. This fetches the most recent summary and messages which are not summarized yet.
    def get_chat_history(self, user_id):
        

        # Retrieve the summary for the user and add it to the history
        summary_key = f"{user_id}:summary"
        summary = self.redis_client.get(summary_key)
        history = []
        
        if summary is not None:
            history.append(json.loads(summary))

        # Retrieve all messages from the Redis stack which have not been summarized. This batch will be summarized when it reaches 20 messages.
        # Combine the remaining messages with the summary
        stack_key = f"{user_id}:stack"
        remaining_messages = [json.loads(msg) for msg in self.redis_client.lrange(stack_key, 0, -1)]
        history.extend(remaining_messages)

        return history

    # We use this method to load the last 20 messages for a user from DynamoDB and populate the Redis cache. This is useful when the cache is reset or when the application is restarted.
    def load_messages_from_dynamodb(self, user_id):
       
        table = self.dynamodb_resource.Table(TABLE_NAME)
        
        # Query the table with a limit of 20 items
        response = table.query(
            KeyConditionExpression=Key('UserId').eq(user_id),
            Limit=20,
            ScanIndexForward=False # Retrieve in descending order of the sort key: Timestamp
        )

        # Reset Redis state and populate messages
        stack_key = f"{user_id}:stack"
        summary_key = f"{user_id}:summary"
        self.redis_client.delete(stack_key)
        self.redis_client.delete(summary_key)
        
        # Rehydrate redis cache
        for item in reversed(response['Items']):
            print("Reloading:"+str(item))
            message = {"batch_id": int(item['BatchId']), "content": item['Content'], "timestamp": item['Timestamp']}
            self.redis_client.rpush(stack_key, json.dumps(message))




Next we instantiate the class. To demonstrate the new messages and summarization workflows, we load sample conversations. Please check sample_conversations.py for a sample chat history.

In [ ]:
import pprint

chat_manager = ChatManager(redis_host, redis_port, dynamodb_table, REGION_NAME)


Load Sample Messages. Summarization worflow gets triggered when we load 20 messages.

In [ ]:
from sample_conversation import conversation

for c in conversation:
    chat_manager.handle_new_message("user1", str(c))

Now let's fetch the chat history at this point and youll see that it fetches most recent messages which have not been summarized (because the batch did not reach count 20) and the most recent summary. Notice the bacth id.

In [ ]:
history = chat_manager.get_chat_history("user1")
pprint.pprint(history)

Now we use a loop to create a simple chatbot. On every new message, we first fetch the history, combine it with the question and send it to the LLM to get a response. Once we get a response, we add the user's message and LLM's response to Redis and DynamoDB accordingly using handle_new_message function.

In [ ]:
question_prompt = """
Given a question and chat history, answer the question in the context of the conversation.

Chat History:
{chat_history}

Question: {question}
"""



for i in range(6):
    question = input()
    message = {"role":"user", "content": question}
    history = str(chat_manager.get_chat_history("user1"))
    prompt = question_prompt.format(chat_history=history, question=question)
    response = call_bedrock_claude_3(prompt, 0.7)
    chat_manager.handle_new_message("user1", str(message))
    chat_manager.handle_new_message("user1", str(response))
    print(response)
    

We can now see the chat history again

In [ ]:
history = chat_manager.get_chat_history("user1")
pprint.pprint(history)

If the cache is deleted and we want to rehydrate cache with previous 20 messages, we can call the below method

In [ ]:
chat_manager.load_messages_from_dynamodb('user1')